### Perfilamiento y limpieza de datos - BI, Proyecto 2, Entrega 3
Se genera un reporte sobre los datos. En lo que es relevante al contexto, interesa el número de registros vacías, datos más frecuentes, si es una variable nominal o numérica, si tiene alta cardinalidad, información estadística sobre la distribución de los datos, y número de registros únicos.

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import pandas_profiling as pd_pf

In [4]:
# Se lee la sheet v2 que contiene las correcciones mencionadas previamente
df1 = pd.read_excel("Recaudo y Trafico en Peajes 2014-2018 archivo final.xlsx", sheet_name="Tabla Recaudo 2014-2018 final")
df2 = pd.read_excel("Recaudo y Trafico en Peajes 2014-2018 archivo final.xlsx", sheet_name="Tabla Trafico 2014-2018 final")

In [5]:
df1.head(1)

,No.,Codigo Invias,SiglaProyecto,NombreProyecto,NombreEstacionPeaje,2014-01-01 00:00:00,2014-02-01 00:00:00,2014-03-01 00:00:00,2014-04-01 00:00:00,2014-05-01 00:00:00,...,2018-03-01 00:00:00,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,2018-11-01 00:00:00,2018-12-01 00:00:00
0,99.0,3846.0,P3,AUTOPISTA CONEXIÓN PACIFICO 3,ACAPULCO,0.0,0.0,0.0,0.0,0.0,...,1430949000,1107747800,1217613100,1584564500,1925055900,1927190700,1712171500,1859298900,1812760800,1925974100


In [8]:
df2.head(1)

,No.,Codigo Invias,SiglaProyecto,NombreProyecto,NombreEstacionPeaje,2014-01-01 00:00:00,2014-02-01 00:00:00,2014-03-01 00:00:00,2014-04-01 00:00:00,2014-05-01 00:00:00,...,2018-03-01 00:00:00,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,2018-11-01 00:00:00,2018-12-01 00:00:00
0,99,3846.0,P3,AUTOPISTA CONEXIÓN PACIFICO 3,ACAPULCO,0.0,0.0,0.0,0.0,0.0,...,129099,109589,120429,112631,121432,119666,106213,115864,115550,129337


In [10]:
df1.columns

Index([                'No.',       'Codigo Invias',       'SiglaProyecto',
            'NombreProyecto', 'NombreEstacionPeaje',   2014-01-01 00:00:00,
         2014-02-01 00:00:00,   2014-03-01 00:00:00,   2014-04-01 00:00:00,
         2014-05-01 00:00:00,   2014-06-01 00:00:00,   2014-07-01 00:00:00,
         2014-08-01 00:00:00,   2014-09-01 00:00:00,   2014-10-01 00:00:00,
         2014-11-01 00:00:00,   2014-12-01 00:00:00,   2015-01-01 00:00:00,
         2015-02-01 00:00:00,   2015-03-01 00:00:00,   2015-04-01 00:00:00,
         2015-05-01 00:00:00,   2015-06-01 00:00:00,   2015-07-01 00:00:00,
         2015-08-01 00:00:00,   2015-09-01 00:00:00,   2015-10-01 00:00:00,
         2015-11-01 00:00:00,   2015-12-01 00:00:00,   2016-01-01 00:00:00,
         2016-02-01 00:00:00,   2016-03-01 00:00:00,   2016-04-01 00:00:00,
         2016-05-01 00:00:00,   2016-06-01 00:00:00,   2016-07-01 00:00:00,
         2016-08-01 00:00:00,   2016-09-01 00:00:00,   2016-10-01 00:00:00,
         201

### Limpieza de datos
Se remueven tildes y duplicados de los DataFrame que se usarán luego para crear la tabla de hechos.

In [44]:
prev = [len(df1.index), len(df2.index)]
cols = df1.select_dtypes(include=[np.object]).columns
df1[cols] = df1[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df1 = df1.sort_values(by=['NombreEstacionPeaje'])
df1.drop_duplicates(subset='NombreEstacionPeaje', keep=False, inplace=True, ignore_index=True)

cols = df2.select_dtypes(include=[np.object]).columns
df2[cols] = df2[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df2 = df2.sort_values(by=['NombreEstacionPeaje'])
df2.drop_duplicates(subset='NombreEstacionPeaje', keep=False, inplace=True, ignore_index=True)

print(f"Para df1 y df2, antes: {prev[0]} y {prev[1]} columnas, ahora {len(df1.index)} y {len(df2.index)}, respectivamente.")

Para df1 y df2, antes: 118 y 117 columnas, ahora 118 y 117, respectivamente.


### Tabla de fechas
El formato de Cod_fecha es 20XX_YY, para XX = {14,...,18} y YY = {01, 02,..., 12}.

In [31]:
x = []
c = df1.columns
meses = ['Enero','Febreo','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
m = 0;
for i in range(5,len(c)):
    cod_fecha = pd.to_datetime(df1.columns[i]).strftime('%Y_%m')
    anio = cod_fecha[0:4]
    mes = meses[m]
    semestre = 1 if m<6 else 2
    cuad = 1 if m < 4 else (2 if m <8 else 3)
    tri = 1 if m < 3 else (2 if m <6 else (3 if m <9 else 4))
    x.append([cod_fecha,anio,semestre,cuad,tri,mes])
    m = (m+1)%12

fechas = pd.DataFrame(x, columns = ['Cod_fecha', 'Anio', 'Semestre', 'Cuadrimestre', 'Trimestre', 'Mes'])
fechas.to_excel("fechas_final.xlsx")
fechas.tail()

,Cod_fecha,Anio,Semestre,Cuadrimestre,Trimestre,Mes
55,2018_08,2018,2,2,3,Agosto
56,2018_09,2018,2,3,3,Septiembre
57,2018_10,2018,2,3,4,Octubre
58,2018_11,2018,2,3,4,Noviembre
59,2018_12,2018,2,3,4,Diciembre


### Tabla de hechos
Se genera la tabla de hechos de forma que se tenga una el nombre de la estación de peaje, su recaudo, tráfico y código fecha respectivo (asignado previamente en la tabla de fechas).

In [53]:
def split_fechas(df_recaudo, df_trafico):
    x = []
    c = df_recaudo.columns

    for index, row in df_recaudo.iterrows():
        for j in range(len(row)):
            if j > 4:
                x.append([row[0], row[1], row[2], row[3], row[4], row[j]])
    i = 0
    for index, row in df_trafico.iterrows():
        for j in range(len(row)):
            if j > 4:
                x[i].append(row[j])
                x[i].append(pd.to_datetime(df_trafico.columns[j]).strftime('%Y_%m'))
                i += 1
    df_f = pd.DataFrame(x, columns = ['No.', 'Codigo Invias', 'SiglaProyecto',
                                      'NombreProyecto', 'NombreEstacionPeaje',
                                      'Recaudo', 'Trafico', 'Cod_fecha'])
    df_f.drop(columns=['No.', 'Codigo Invias', 'SiglaProyecto', 'NombreProyecto'], inplace=True)
    return df_f

df_final = split_fechas(df1, df2)
df_final.to_excel("Tabla_de_hechos_final.xlsx")
df_final.head()

,NombreEstacionPeaje,Recaudo,Trafico,Cod_fecha
0,ACAPULCO,0.0,0.0,2014_01
1,ACAPULCO,0.0,0.0,2014_02
2,ACAPULCO,0.0,0.0,2014_03
3,ACAPULCO,0.0,0.0,2014_04
4,ACAPULCO,0.0,0.0,2014_05
